In [1]:
import matplotlib.pyplot as mplt
import pandas as pd
import numpy as np
import geopandas as gpd

import cProfile
import pstats

from functools import partial
from types import MappingProxyType
#from tqdm import tqdm

from helper import DataHandler, Plot, plot_grid
from functools import partial
from functools import partial

from partition import Partition, Tally, Assignment
from grid import Grid
from markovchain import SingleMetricOptimizer, hierarchical_recom
import constraints

In [2]:
"Load data"

handler = DataHandler()
graph = handler.load_graph()
chicago = handler.load_chicago()
candidates = handler.load_candidates()  # set of nodes
geo_centers = handler.load_geo_centers()
geo_candidates = handler.load_geo_candidates()
#travel_times_walking = handler.load_travel_walking()
#travel_times = handler.load_travel_times() # travel time between blocks is 10 minutes
newtravel = handler.load_newtravel()
geometries = handler.load_geometries()
plt = Plot()

In [3]:
from graph import Graph
import networkx as nx

"""Returns a tree that looks like this:

1 - 2 - 3 - 4
    |       |
5 - 6 - 7   8 - 9 - 10
|       |       |
11-12   13      14
|
15-16-17
|
18-19
|
20

"""
tree = Graph()
tree.add_edges_from(
    [
        (1, 2),(2, 3),(3, 4),
        (2, 6),(4, 8),
        (5, 6),(6, 7),(8, 9),(9, 10),
        (5, 11),(7, 13),(9, 14),
        (11, 12),
        (11, 15),
        (15, 16),(16, 17),
        (15, 18),
        (18, 19),
        (18, 20),
    ]
)    


In [11]:
def find_successors(tree, root):
    return {a: b for a, b in nx.bfs_successors(tree, root)}
    

succ = find_successors(tree, 2)

from tree import _part_nodes, compute_subtree_nodes

In [22]:
from tree import random_spanning_tree

random_tree = random_spanning_tree(graph)

In [23]:
part_nodes = {}

for node in random_tree.nodes:
    part_nodes[node] = _part_nodes(succ, node)

In [ ]:
part_nodes

In [24]:
nodes = compute_subtree_nodes(random_tree, succ, 2)
nodes

{1: {1},
 10: {10},
 14: {14},
 9: {9, 10, 14},
 8: {8, 9, 10, 14},
 4: {4, 8, 9, 10, 14},
 3: {3, 4, 8, 9, 10, 14},
 12: {12},
 17: {17},
 16: {16, 17},
 19: {19},
 20: {20},
 18: {18, 19, 20},
 15: {15, 16, 17, 18, 19, 20},
 11: {11, 12, 15, 16, 17, 18, 19, 20},
 5: {5, 11, 12, 15, 16, 17, 18, 19, 20},
 13: {13},
 7: {7, 13},
 6: {5, 6, 7, 11, 12, 13, 15, 16, 17, 18, 19, 20},
 2: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20}}

In [ ]:
grid_opposite = Grid(dimensions=(20,20), num_candidates=50, density='opposite', threshold=(10,10))
graph_opposite = grid_opposite.graph
plot_grid(grid_opposite.graph)

In [ ]:
# Define a wrapper function for profiling
def run_partition(newtravel):
    "Initial partition"
    pop = sum(graph.nodes[node]['pop'] for node in graph.nodes)
    #area = sum(graph.nodes[node]['area'] for node in graph.nodes)   
    #avg_density = area / pop
    # nodelarin density check i burda yapilip sonuc attribute olarak atansin. Attribute ismi simdilik density
    #check_density = ?
    Assignment.travel_times = MappingProxyType(newtravel)
    column_names = ['pop', 'area', 'real_phc', 'avg_density']
    
    initial_solution = Partition.from_random_assignment(
    graph=graph, 
    assignment_class=Assignment,
    capacity_level = 1,
    epsilon=0.01,
    pop_target = pop // 100,
    column_names = column_names,
    #density = 0.5
    )
    return initial_solution


cProfile.run('run_partition(newtravel)', 'profile_output')


p = pstats.Stats('profile_output')
p.strip_dirs()           # Optional: clean up file paths for readability
p.sort_stats('cumulative')  # Sort by cumulative time or choose 'time', 'calls', etc.
p.print_stats(20)  

In [ ]:
"Initial partition"
pop = sum(graph.nodes[node]['population'] for node in graph.nodes)
#area = sum(graph.nodes[node]['area'] for node in graph.nodes)   
#avg_density = area / pop
# nodelarin density check i burda yapilip sonuc attribute olarak atansin. Attribute ismi simdilik density
#check_density = ?
Assignment.travel_times = MappingProxyType(newtravel)
column_names = ['population', 'area', 'candidate', 'density']

initial_solution = Partition.from_random_assignment(
graph=graph, 
assignment_class=Assignment,
capacity_level = 1,
epsilon=0.005,
pop_target = pop // 100,
column_names = column_names,
#density = 0.5
)

In [ ]:
"Plot initial partition"
#m, regions, chicago, geocenters = initial_solution.plot_map(attr="district")
#m

"Plot side by side"
fig, regions_initial_new, regions_final_new, centers, others = plt.compare(initial_solution, initial_solution)
fig

In [16]:
m.save("map0.html")

In [ ]:

"Proposal and constraints"   # See how we call this in optimizer. 
proposal = partial(
    hierarchical_recom,
    pop_target = pop // 100,
    column_names = column_names,
    epsilon=0.02,
    density = None)
constraints = constraints.within_percent_of_ideal_population(initial_solution)

In [5]:
"Optimizer"
 
#sum_travel_radius = lambda p: np.sum(p.radius.values())
#num_cut_edges = lambda p: len(p["cut_edges"])
average_radius = lambda p: np.sum(((np.array(list(p.radius.values())) - np.sum(list(p.radius.values())) / len(p))**2))

optimizer = SingleMetricOptimizer(
    proposal=proposal,
    constraints=constraints,
    initial_state=initial_solution,
    optimization_metric=average_radius,
    maximize=False)

In [ ]:
"Tilted Runs"

total_steps = 1000
partitions = {}

min_scores_tilt = np.zeros(total_steps)
for i, part in enumerate(optimizer.tilted_run(total_steps, p=0.125, with_progress_bar=True)):
    min_scores_tilt[i] = optimizer.best_score
    partitions[i] = optimizer.best_part
final_partition = optimizer.best_part


In [ ]:
initial_solution['population']

In [ ]:
"Plot side by side"
fig, regions_initial_new, regions_final_new, centers, others = plt.compare(initial_partition, final_partition)
fig


# Last result for average radius with fixed travel time

In [ ]:
import matplotlib.pyplot as mplt
fig, ax = mplt.subplots(figsize=(12,6))
mplt.plot(min_scores_tilt, label="Tilted Run")
mplt.xlabel("Steps", fontsize=20)
mplt.ylabel("Sum of radius_average", fontsize=20)
mplt.legend()
mplt.show()

In [ ]:
"Plot side by side"
fig, regions_initial_new, regions_final_new, centers, others = plt.compare(initial_partition, final_partition)
fig

In [ ]:
#sum_travel_radius = lambda p: np.sum(p.radius.values())
num_cut_edges = lambda p: len(p["cut_edges"])
average_radius = lambda p: np.sum(abs((np.array(list(p.radius.values())) - np.sum(list(p.radius.values())) / len(p))**2))

In [ ]:
import gerrytools

import matplotlib.pyplot as plt
from gerrytools.scoring import *
from gerrytools.plotting import *
import gerrytools.plotting.colors as colors
import numpy as np

N = len(regions_final_new)

dists = regions_final_new.to_crs("EPSG:3857")
dists["final_district"] = dists["final_district"].astype(int)
dists=dists.sort_values(by="final_district")
dists["colorindex"] = list(range(N))
dists["color"] = colors.districtr(N)

ax = drawplan(chicago, assignment="final_district", overlays=None)

In [ ]:
import maup
from maup import repair

In [ ]:
maup.doctor(chicago)

In [ ]:
repair.count_holes(chicago)

In [ ]:
maup.repair.autorepair(chicago)

In [ ]:
final_partition.plot(chicago, figsize=(10, 10), cmap="tab20")
mplt.axis('off')
mplt.show()

In [ ]:
import numpy as np
print("1) Number of Cut Edges")
print("    Best score: ", optimizer.best_score)
print("    Initial score: ", len(initial_partition["cut_edges"]))
print("2) Sum of Travel Radius")
print("    Initial: ", np.sum(list(initial_partition.radius.values())))
print("    Final: ", np.sum(list(final_partition.radius.values())))
print("3) Sum of Radius Devitation")
print("    Initial: ", sum(abs(np.array(list(initial_partition.radius.values())) - sum(x for x in initial_partition.radius.values()) / len(initial_partition))**2))
print("    Final: ", sum(abs(np.array(list(final_partition.radius.values())) - sum(x for x in final_partition.radius.values())/ len(final_partition))**2 ))

In [ ]:
"""pcompress
radiuss = {}
i=0
for optimizer in Record(optimizer.tilted_run(total_steps, p=0.125, with_progress_bar=True), "pa-run.chain"):
    # normal chain stuff here
    partition = optimizer.best_part
    radius = partition.radius
    radiuss[i] = radius 
    i+= 1"""

In [ ]:
"Watch"
%matplotlib inline
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('png')

import pandas as pd

import matplotlib.cm as mcm
import matplotlib.pyplot as mplt
import networkx as nx
from PIL import Image
import io
import ipywidgets as widgets
from IPython.display import display, clear_output

frames = []
district_data = []

for i, partition in enumerate(recom_chain):
    for district_name in partition.perimeter.keys():
        population = partition.population[district_name]
        perimeter = partition.perimeter[district_name]
        area = partition.area[district_name]
        district_data.append((i, district_name, population, perimeter, area))

    buffer = io.BytesIO()
    fig, ax = plt.subplots(figsize=(10,10))
    partition.plot(ax=ax, cmap='tab20')
    ax.set_xticks([])
    ax.set_yticks([])
    plt.savefig(buffer, format='png', bbox_inches='tight')
    buffer.seek(0)
    image = Image.open(buffer)
    frames.append(image)
    plt.close(fig)

df = pd.DataFrame(
    district_data,
    columns=[
        'step',
        'district_name',
        'population',
        'perimeter',
        'area'
    ]
)

def show_frame(idx):
    clear_output(wait=True)
    display(frames[idx])

slider = widgets.IntSlider(value=0, min=0, max=len(frames)-1, step=1, description='Frame:')
slider.layout.width = '500px'
widgets.interactive(show_frame, idx=slider)
#df.head(5)
#The perimeter and area attributes are actually not present in the MN_precincts.geojson file, but the GeographicPartition class will calculate them at instantiation time using the geometries provided in the file.

# Save

In [ ]:
#final_partition = optimizer.best_part
#for partition in pcompress.Record(optimizer, "run.chain", executable="pv", extreme=True):
#    print(partition.population)

#final_partition = optimizer.best_part
#handler = DataHandler()
#handler.load_final_assignment()
#final_partition = optimizer.best_part
#final_assignment = dict(final_partition.assignment)
#pd.to_pickle(final_assignment, '/Users/kirtisoglu/Documents/GitHub/Allocation-of-Primary-Care-Centers-in-Chicago/prepared_data/final_assignment.pkl')
#final_assignment = handler.load_final_assignment()
#final_assignment = Partition(graph, final_assignment, updaters=my_updaters)

# Local Search

In [ ]:
# We can run each of the optimization methods and collect some data

total_steps = 10000

# Short Bursts
min_scores_sb = np.zeros(total_steps)
for i, part in enumerate(optimizer.short_bursts(5, 2000, with_progress_bar=True)):
    min_scores_sb[i] = optimizer.best_score

# Simulated Annealing
min_scores_anneal = np.zeros(total_steps)
for i, part in enumerate(
    optimizer.simulated_annealing(
        total_steps,
        optimizer.jumpcycle_beta_function(200, 800),
        beta_magnitude=1,
        with_progress_bar=True
    )
):
    min_scores_anneal[i] = optimizer.best_score